In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import sys,os
import numpy as np
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.collections as pltc
import logging

import shapely
import meshpy.triangle

import workflow.conf
import workflow.download
import workflow.triangulate
import workflow.warp
import workflow.plot
import workflow.tree
import workflow.hucs
import workflow.hydrography

import vtk_io # from ATS/tools/meshing_ats



In [3]:
simplify_distance = 10 # meters

myhuc = '06010208'
outdir = "data/meshes/%s/12"%myhuc
if not os.path.isdir(outdir):
    os.makedirs(outdir)



In [4]:
# deal with logging to stdout
root = logging.getLogger()
root.setLevel(logging.INFO)
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)
 

In [5]:
## === Preprocess HUCs ===
logging.info("Preprocessing HUCs")
logging.info("=====================")

# collect HUC shapefile
logging.info("downloading HUC %s"%myhuc[0:2])
workflow.download.download_huc(myhuc[0:2])

# load shapefiles for the HUC8 of interest
logging.info("loading HUC %s"%myhuc)
profile, huc8 = workflow.conf.load_huc(myhuc)

# load shapefiles for all HUC 12s in the Obed HUC 8.
logging.info("loading all 12s")
profile, huc12s = workflow.conf.load_hucs_in('06010208', 12)

# change coordinates to meters (in place)
logging.info("change coordinates to m")
for huc12 in huc12s:
    workflow.warp.warp_shape(huc12, profile['crs'], workflow.conf.default_crs())

# convert to shapely
huc_shapes = [shapely.geometry.shape(s['geometry']) for s in huc12s]

# split
logging.info("Split form HUCs")
hucs = workflow.hucs.HUCs(huc_shapes)
logging.info("...done")

2018-07-16 10:51:48,750 - root - INFO - Preprocessing HUCs
2018-07-16 10:51:48,753 - root - INFO - =====================
2018-07-16 10:51:48,754 - root - INFO - downloading HUC 06
2018-07-16 10:51:48,755 - root - INFO - Collecting HUC: "06"
2018-07-16 10:51:48,763 - root - INFO - loading HUC 06010208
2018-07-16 10:51:48,913 - root - INFO - loading all 12s
2018-07-16 10:51:49,143 - root - INFO - change coordinates to m
2018-07-16 10:51:49,174 - root - INFO - Split form HUCs
2018-07-16 10:51:50,349 - root - INFO - ...done


In [6]:
## === Preprocess hydrography ===
logging.info("Preprocessing hydrography")
logging.info("==========================")

# collect hydrography
logging.info("downloading Hydrography %s"%myhuc)
workflow.download.download_hydro(myhuc)

# load stream network
logging.info("loading streams")
rprofile, rivers = workflow.conf.load_hydro('06010208')

# change coordinates to meters (in place)
logging.info("change coordinates to m")
for river in rivers:
    workflow.warp.warp_shape(river, rprofile['crs'], workflow.conf.default_crs())

# convert to shapely
logging.info("merging reaches")
rivers_s = shapely.geometry.MultiLineString([shapely.geometry.shape(r['geometry']) for r in rivers])
rivers_s2 = shapely.ops.linemerge(rivers_s).simplify(1.e-5)

logging.info("forming global tree")
rivers = workflow.hydrography.make_global_tree(rivers_s2, tol=0.1)
del rivers_s
del rivers_s2
logging.info("...done")

2018-07-16 10:51:50,434 - root - INFO - Preprocessing hydrography
2018-07-16 10:51:50,437 - root - INFO - ==========================
2018-07-16 10:51:50,442 - root - INFO - downloading Hydrography 06010208
2018-07-16 10:51:50,446 - root - INFO - Collecting HUC Hydrography: "06010208"
2018-07-16 10:51:50,447 - root - INFO - loading streams
2018-07-16 10:51:50,865 - root - INFO - change coordinates to m
2018-07-16 10:51:51,194 - root - INFO - merging reaches
2018-07-16 10:51:52,158 - root - INFO - forming global tree
2018-07-16 10:51:52,563 - root - INFO - ...done


In [7]:
# a few small trees can be removed
for i in reversed(range(len(rivers))):
    ltree = len(rivers[i])
    if ltree < 10:
        rivers.pop(i)
    else:
        print("keeping one of length", ltree)

keeping one of length 5473


In [8]:
# simplify and prune
logging.info("simplify and prune rivers")
workflow.hydrography.cleanup(rivers, simplify_distance, simplify_distance, simplify_distance)
logging.info("simplify HUCs")
workflow.hucs.simplify(hucs, simplify_distance)
logging.info("...done")

2018-07-16 10:51:52,745 - root - INFO - simplify and prune rivers
2018-07-16 10:51:53,287 - root - INFO -     cleaned inner segment of length 9.4836 at centroid (980835.1804743886, 1510148.8389611207)
2018-07-16 10:51:53,292 - root - INFO -     cleaned inner segment of length 9.00602 at centroid (986906.9574483884, 1503163.3019694358)
2018-07-16 10:51:53,301 - root - INFO -     cleaned inner segment of length 3.67397 at centroid (975519.8083108478, 1490668.9926833385)
2018-07-16 10:51:53,311 - root - INFO -     cleaned inner segment of length 8.3225 at centroid (977282.7242552503, 1476609.474854385)
2018-07-16 10:51:53,318 - root - INFO -     cleaned inner segment of length 7.76292 at centroid (1008015.8491485287, 1521036.1073649758)
2018-07-16 10:51:53,324 - root - INFO -     cleaned inner segment of length 8.61727 at centroid (1002954.0880678874, 1499733.8393994668)
2018-07-16 10:51:53,333 - root - INFO - simplify HUCs
2018-07-16 10:51:53,430 - root - INFO - ...done


In [9]:
# snap
logging.info("snap")
workflow.hydrography.snap(hucs, rivers, simplify_distance, 10*simplify_distance)

2018-07-16 10:51:53,492 - root - INFO - snap
2018-07-16 10:51:53,494 - root - INFO -  Snapping polygon segment boundaries to river endpoints
2018-07-16 10:51:53,934 - root - INFO -  Snapping river endpoints to the polygon
2018-07-16 10:51:59,587 - root - INFO -   - snapped river: (1010661.231952778, 1507594.537775388) to (1010661.231952778, 1507594.537775388)
2018-07-16 10:51:59,595 - root - INFO -   - snapped river: (1010661.231952778, 1507594.537775388) to (1010661.231952778, 1507594.537775388)
2018-07-16 10:51:59,600 - root - INFO -   - snapped river: (1010661.231952778, 1507594.537775388) to (1010661.231952778, 1507594.537775388)
2018-07-16 10:51:59,630 - root - INFO -   - snapped river: (1010661.231952778, 1507594.537775388) to (1010661.231952778, 1507594.537775388)
2018-07-16 10:51:59,646 - root - INFO -   - snapped river: (1010661.231952778, 1507594.537775388) to (1010661.231952778, 1507594.537775388)
2018-07-16 10:51:59,657 - root - INFO -   - snapped river: (1010661.231952778,

True

In [ ]:
workflow.plot.rivers(rivers, color='g', style='-x')
workflow.plot.hucs(hucs, color='k', style='-.x')

In [10]:
mins = []
for river in rivers:
    for line in river.dfs():
        coords = np.array(line.coords[:])
        dz = np.linalg.norm(coords[1:] - coords[:-1], 2, -1)
        if np.min(dz) < 10:
            plt.plot(coords[:,0], coords[:,1], 'r-x')
        mins.append(np.min(dz))
print("min min:", min(mins))
print("median:", np.median(np.array(mins)))

plt.show()


min min: 10.147029616163628
median: 60.57112798822514


In [11]:
mins = []
for line in hucs.segments:
    coords = np.array(line.coords[:])
    dz = np.linalg.norm(coords[1:] - coords[:-1], 2, -1)
    mins.append(np.min(dz))
print("min min:", min(mins))
print("median:", np.median(np.array(mins)))    

min min: 10.816764915509692
median: 26.81410863382822


In [17]:
from importlib import reload
reload(workflow.triangulate)
reload(workflow.tree)

<module 'workflow.tree' from '/Users/uec/research/water/data/meshing/workflow/tree.py'>

In [ ]:
# === triangulate ===
logging.info("triangulate")
#refine_func = workflow.triangulate.refine_from_river_distance(500, 100, 1000, 500, rivers)
refine_func = workflow.triangulate.refine_from_max_area(1.e6)
mesh_points, mesh_tris = workflow.triangulate.triangulate(hucs, rivers, verbose=True, refinement_func=refine_func)

# plot
plt.figure()
workflow.plot.tri(mesh_points, mesh_tris)
plt.show()

2018-07-16 10:56:57,819 - root - INFO - triangulate
2018-07-16 10:56:57,821 - root - INFO - Triangulating...
2018-07-16 10:56:58,107 - root - INFO -    37534 points and 38069 facets
